<a href="https://colab.research.google.com/github/RosetheOnly/codequeens-AI-model/blob/main/CodeQueensAImodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from faker import Faker
import random

# Set seed for reproducibility
np.random.seed(42)
random.seed(42)

# Initialize Faker
fake = Faker()

# 1. Generate classification data
X, y = make_classification(
    n_samples=1000,
    n_features=5,
    n_informative=3,
    n_redundant=0,
    n_classes=2,
    random_state=42
)

# 2. Create DataFrame
df = pd.DataFrame(X, columns=[
    'amount_log',       # Will be converted to actual M-Pesa amount
    'balance_change',
    'time_since_last',
    'merchant_score',
    'location_score'
])
df['Is_Fraud'] = y

# 3. Add user info
users = [f'USER_{i+1}' for i in range(10)]
df['Account'] = np.random.choice(users, size=1000)

# 4. Simulate timestamps
timestamps = pd.date_range(
    start='2025-07-01',
    periods=1000,
    freq='2min'
)
df['Date'] = np.random.choice(timestamps, size=1000)

# 5. Convert amount_log to realistic values
df['Transaction_Amount'] = (10 ** df['amount_log'].clip(-1, 4)).astype(int)

# 6. Add transaction types
transaction_types = ['Send Money', 'Buy Goods', 'Pay Bill', 'Withdraw', 'Deposit']
df['Transaction_Type'] = np.random.choice(transaction_types, size=1000)

# 7. Add recipient phone numbers
df['Recipient_Number'] = [fake.phone_number() for _ in range(1000)]

# 8. Compute running balance per user
df = df.sort_values(['Account', 'Date'])
df['Wallet_Impact'] = df['balance_change'] * df['Transaction_Amount'] / 100
df['Wallet_Impact'] = df['Wallet_Impact'].round(2)

# Simulate starting balance and compute running balance
df['Balance'] = df.groupby('Account')['Wallet_Impact'].cumsum() + 1000  # start with 1000 KES

# 9. Final cleanup
final_cols = [
    'Account', 'Date', 'Transaction_Type', 'Transaction_Amount',
    'Wallet_Impact', 'Balance', 'Recipient_Number', 'Is_Fraud'
]
df_final = df[final_cols].sort_values('Date').reset_index(drop=True)

# 10. Save to CSV
df_final.to_csv('synthetic_mpesa_transactions.csv', index=False)

print("✅ Synthetic M-Pesa transaction data saved to 'synthetic_mpesa_transactions.csv'")


✅ Synthetic M-Pesa transaction data saved to 'synthetic_mpesa_transactions.csv'


In [2]:
%pip install faker -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import timedelta

# Initialize
fake = Faker()
np.random.seed(42)
random.seed(42)

# Configuration
users = [f'USER_{i+1}' for i in range(10)]
months = pd.date_range(start='2024-08-01', periods=12, freq='MS')
providers = {
    'KPLC': {'unit': 'kWh', 'min': 30, 'max': 450, 'rate': 22.0},
    'Nairobi Water': {'unit': 'm³', 'min': 5, 'max': 50, 'rate': 80.0}
}

records = []

for user in users:
    phone = fake.phone_number()
    for month in months:
        for provider, config in providers.items():
            units = round(np.random.uniform(config['min'], config['max']), 1)
            base = units * config['rate']
            fee = np.random.uniform(50, 150)
            total = round(base + fee, 2)

            # Simulate billing and payment dates
            billing_date = pd.to_datetime(month)
            due_date = billing_date + pd.Timedelta(days=14)
            actual_pay_date = due_date + pd.Timedelta(days=random.choice([0, 0, 2, 5, 10]))  # most pay on time

            # Determine if late
            was_late = actual_pay_date > due_date
            penalty = round(total * 0.05, 2) if was_late else 0.00
            final_amount = round(total + penalty, 2)

            # Simulate M-Pesa payment
            mpesa_code = fake.bothify(text='???###??')
            mpesa_time = actual_pay_date + timedelta(hours=random.randint(1, 8))

            records.append({
                'User_ID': user,
                'Provider': provider,
                'Billing_Date': billing_date.date(),
                'Due_Date': due_date.date(),
                'Units_Used': units,
                'Unit_Type': config['unit'],
                'Total_Bill_KSh': total,
                'Was_Late': was_late,
                'Penalty_KSh': penalty,
                'Final_Amount_KSh': final_amount,
                'Paid_On': actual_pay_date.date(),
                'M-Pesa_Code': mpesa_code.upper(),
                'Phone_Number': phone,
                'Payment_Time': mpesa_time.strftime('%Y-%m-%d %H:%M:%S')
            })

# Create DataFrame
df = pd.DataFrame(records)

# Save to CSV
df.to_csv('utility_bills_with_mpesa.csv', index=False)
print("✅ CSV with M-Pesa confirmations and penalties saved as 'utility_bills_with_mpesa.csv'")

✅ CSV with M-Pesa confirmations and penalties saved as 'utility_bills_with_mpesa.csv'


In [7]:
import pandas as pd

# Load the utility bill data from the CSV into a DataFrame
try:
    utility_df = pd.read_csv('utility_bills_with_mpesa.csv')
except FileNotFoundError:
    print("Error: 'utility_bills_with_mpesa.csv' not found. Please ensure the file is generated.")
    utility_df = pd.DataFrame() # Create an empty DataFrame to avoid further errors

print("utility_bill_with_mpesa columns:", utility_df.columns.tolist())

utility_bill_with_mpesa columns: ['User_ID', 'Provider', 'Billing_Date', 'Due_Date', 'Units_Used', 'Unit_Type', 'Total_Bill_KSh', 'Was_Late', 'Penalty_KSh', 'Final_Amount_KSh', 'Paid_On', 'M-Pesa_Code', 'Phone_Number', 'Payment_Time']


In [8]:
# 🛠️ Step 1: Install Required Libraries
!pip install pandas PyMuPDF

# 📦 Step 2: Import Libraries
import pandas as pd
import zipfile
import os
from pathlib import Path
import re
import fitz  # PyMuPDF for PDF parsing

# 📁 Step 3: Mount Google Drive (if your files are there)
# from google.colab import drive
# drive.mount('/content/drive')

# 📂 Step 4: Set Paths to Your Files (update if needed)
mpesa_csv = '/content/synthetic_mpesa_transactions.csv'
bills_csv = '/content/utility_bills_with_mpesa.csv'
bank_zip = '/content/fwdmockbankstatements.zip'
extract_dir = '/content/unzipped_pdfs'

# 📤 Step 5: Extract Bank Statements from ZIP
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(bank_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Define the bank statement parsing function (corrected from BWA3Kxh_ZOl0)
def improved_parse_bank_statement_text(text):
    """
    Parses text from a bank statement PDF to extract transaction data.
    Now includes logic to extract Account Number and handle varied formats.
    """
    transactions = []
    account_number = "Unknown"

    # Extract Account Number (Handling different possible patterns)
    acct_match1 = re.search(r'Account Number:\s*([\d\-]+)', text)
    acct_match2 = re.search(r'Account:\s*([\d\-]+)', text)
    if acct_match1:
        account_number = acct_match1.group(1).strip()
    elif acct_match2:
        account_number = acct_match2.group(1).strip()

    # Find the start of the transactions table (Handling different possible headers)
    # Look for 'Date', 'Description', 'Debit', 'Credit', 'Balance' or similar combinations
    # Updated header pattern based on debug output and previous attempts
    # Using \s+ to match one or more whitespace characters, and potentially include newlines
    header_pattern = re.compile(r'Date\s+Description\s+Details\s+Debit\s+\(KES\)\s+Credit\s+\(KES\)\s+Balance\s+\(KES\)', re.IGNORECASE)
    header_match = header_pattern.search(text)

    transaction_text = ""
    if header_match:
        transaction_text = text[header_match.end():]
    else:
        # Fallback: If header not found, try to find patterns of transaction lines
        # This is less reliable and may pick up non-transaction lines
        print(f"Warning: Could not find transaction header in document for account {account_number}. Attempting to parse lines.")
        transaction_text = text # Use the whole text and rely on line parsing regex


    # Regex to find transaction lines (adjust based on observed patterns)
    # This regex is based on the structure seen in the debug output: Date, Description, Details, Debit, Credit, Balance
    # It needs to be flexible to handle varying spaces and missing Debit/Credit values.
    # Assuming Date format DD/MM/YYYY
    # Making whitespace handling more flexible with \s* and handling potential missing columns more robustly
    transaction_line_pattern = re.compile(
        r'^(\d{2}/\d{2}/\d{4})\s*'  # Date (Group 1) - flexible whitespace
        r'(.*?)\s+'               # Description (Group 2 - non-greedy match for anything, followed by one or more spaces)
        r'(.*?)\s*'               # Details (Group 3 - non-greedy match for anything, followed by zero or more spaces)
        r'([\d,\.]*)\s*'          # Debit (Group 4 - includes comma and dot, optional, followed by zero or more spaces)
        r'([\d,\.]*)\s*'          # Credit (Group 5 - includes comma and dot, optional, followed by zero or more spaces)
        r'([\d,\.]+)\s*$', re.MULTILINE # Balance (Group 6 - includes comma and dot, optional trailing spaces, end of line)
    )


    # Process text line by line
    lines = transaction_text.strip().split('\n')
    for line in lines:
         match = transaction_line_pattern.search(line)
         if match:
              date_str, description, details, debit_str, credit_str, balance_str = match.groups()

              # Clean and convert numeric values
              debit = float(debit_str.replace(',', '')) if debit_str and debit_str.strip() else 0.0
              credit = float(credit_str.replace(',', '')) if credit_str and credit_str.strip() else 0.0
              balance = float(balance_str.replace(',', '')) if balance_str and balance_str.strip() else 0.0

              transactions.append({
                  'Account_Number': account_number,
                  'Date': pd.to_datetime(date_str, format='%d/%m/%Y', errors='coerce'),
                  'Description': f"{description.strip()} - {details.strip()}".strip(' -'), # Combine description and details
                  'Debit': debit,
                  'Credit': credit,
                  'Balance': balance
              })


    if not transactions:
        print(f"Warning: No transactions extracted for account {account_number}.")
        return pd.DataFrame() # Return empty DataFrame if no transactions found


    return pd.DataFrame(transactions)



# ✅ Step 6: Clean MPESA Transactions
mpesa = pd.read_csv(mpesa_csv)
mpesa.drop_duplicates(inplace=True)
# Removed string operations from numeric columns
mpesa['Transaction_Amount'] = mpesa['Transaction_Amount'].astype(float)
mpesa['Balance'] = mpesa['Balance'].astype(float)
# Removed string operation from Wallet_Impact
# mpesa['Wallet_Impact'] = mpesa['Wallet_Impact'].str.lower().str.strip() # Wallet_Impact is numeric
mpesa['Date'] = pd.to_datetime(mpesa['Date'], errors='coerce')
mpesa['Source_Type'] = 'MPESA'
# Rename 'Account' to 'Account_Number' for consistency before merging
if 'Account' in mpesa.columns:
    mpesa.rename(columns={"Account": "Account_Number"}, inplace=True)


# ✅ Step 7: Clean Utility Bills
bills = pd.read_csv(bills_csv)
bills.dropna(subset=['Final_Amount_KSh'], inplace=True)
bills['Final_Amount_KSh'] = bills['Final_Amount_KSh'].astype(float)
# Assuming 'Provider' column is intended as the description and is string type
if 'Provider' in bills.columns:
    # bills['Bill_Type'] = bills['Bill_Type'].str.lower().str.strip() # Renamed to Description below
    bills.rename(columns={'Provider': 'Description'}, inplace=True) # Renaming Provider to Description
    if pd.api.types.is_object_dtype(bills['Description']) or pd.api.types.is_string_dtype(bills['Description']):
         bills['Description'] = bills['Description'].astype(str).str.lower().str.strip()
    else:
         print("Warning: 'Description' column in utility bills is not a string type. Skipping string cleaning.")

bills['Date'] = pd.to_datetime(bills['Billing_Date'], errors='coerce').dt.date # Using Billing_Date as the Date for utility bills
# Assuming Balance_After is not present or needs to be calculated later
# bills.rename(columns={'Final_Amount_KSh': 'Amount', 'Balance_After': 'Balance', 'Bill_Type': 'Description'}, inplace=True)
bills.rename(columns={'Final_Amount_KSh': 'Amount'}, inplace=True)
bills['Source_Type'] = 'UTILITY_BILL'
# Rename 'User_ID' to 'Account_Number' for consistency before merging
if 'User_ID' in bills.columns:
    bills.rename(columns={"User_ID": "Account_Number"}, inplace=True)


# ✅ Step 8: Parse Bank PDFs (Using the corrected parsing logic from BWA3Kxh_ZOl0)
# Replicating the parsing loop here for a self-contained cell
all_data = []
if os.path.exists(extract_dir):
    for fname in os.listdir(extract_dir):
        if fname.endswith(".pdf"):
            doc_path = os.path.join(extract_dir, fname)
            try:
                doc = fitz.open(doc_path)
                text = ""
                for page in doc:
                    text += page.get_text()
                # Assuming improved_parse_bank_statement_text takes text as input and returns a DataFrame
                df = improved_parse_bank_statement_text(text) # Need to pass account number if extracted separately
                if not df.empty:
                    all_data.append(df)
            except Exception as e:
                print(f"Error processing {fname}: {e}")

if all_data:
    bank_df = pd.concat(all_data, ignore_index=True)
    bank_df['Source_Type'] = 'BANK_STATEMENT'
    # Ensure 'Date' column in bank_df is datetime
    if 'Date' in bank_df.columns and not pd.api.types.is_datetime64_any_dtype(bank_df['Date']):
        bank_df['Date'] = pd.to_datetime(bank_df['Date'], errors='coerce') # Keep as datetime for merging

else:
    bank_df = pd.DataFrame()
    print("Warning: bank_df is empty. PDF parsing may have failed.")


# ✅ Step 9: Combine All Sources

# Ensure common columns exist and have appropriate names before merging
common_cols_initial = ['Account_Number', 'Date', 'Description', 'Amount', 'Balance', 'Source_Type']

# Select and rename columns for merging
mpesa_clean = mpesa[mpesa.columns.intersection(['Account_Number', 'Date', 'Transaction_Type', 'Transaction_Amount', 'Balance', 'Source_Type'])].copy()
mpesa_clean.rename(columns={'Transaction_Type': 'Description', 'Transaction_Amount': 'Amount'}, inplace=True)
mpesa_clean = mpesa_clean[common_cols_initial].copy() # Reindex to ensure consistent column order and drop extras


bills_clean = bills[bills.columns.intersection(['Account_Number', 'Date', 'Description', 'Amount', 'Source_Type'])].copy()
# Bills data doesn't have a 'Balance' column directly equivalent to account balance
bills_clean['Balance'] = None # Add a placeholder Balance column
# Ensure Date is datetime for merging consistency
if 'Date' in bills_clean.columns and not pd.api.types.is_datetime64_any_dtype(bills_clean['Date']):
    bills_clean['Date'] = pd.to_datetime(bills_clean['Date'], errors='coerce')
bills_clean = bills_clean[common_cols_initial].copy() # Reindex


# Ensure bank_df has the expected columns if populated
bank_clean_merge = pd.DataFrame() # Initialize as empty
if not bank_df.empty:
    bank_clean = bank_df[bank_df.columns.intersection(['Account_Number', 'Date', 'Description', 'Debit', 'Credit', 'Balance', 'Source_Type'])].copy()

    # Create 'Amount' from Debit/Credit for merging purposes
    # Use Credit as positive amount (inflow), Debit as negative amount (outflow)
    bank_clean['Amount'] = bank_clean['Credit'].fillna(0) - bank_clean['Debit'].fillna(0)

    # Select columns for bank_clean for merging
    bank_clean_merge = bank_clean[['Account_Number', 'Date', 'Source_Type', 'Amount', 'Balance', 'Description']].copy()
    # Ensure Date is datetime for merging
    if 'Date' in bank_clean_merge.columns and not pd.api.types.is_datetime64_any_dtype(bank_clean_merge['Date']):
         bank_clean_merge['Date'] = pd.to_datetime(bank_clean_merge['Date'], errors='coerce')
    # Ensure Account_Number exists (should be present if parsing worked)
    if 'Account_Number' not in bank_clean_merge.columns:
         print("Error: 'Account_Number' column is missing in bank_clean_merge after processing. Cannot merge.")
         bank_clean_merge = pd.DataFrame() # Make it empty to prevent merge errors


# Perform Merges
# Concatenate mpesa and bills first
merged_initial = pd.concat([mpesa_clean, bills_clean], ignore_index=True)

# Now merge with bank data only if bank_clean_merge is not empty
if not bank_clean_merge.empty:
    # Ensure date columns are datetime before merging
    if 'Date' in merged_initial.columns and not pd.api.types.is_datetime64_any_dtype(merged_initial['Date']):
         merged_initial['Date'] = pd.to_datetime(merged_initial['Date'], errors='coerce')
    # bank_clean_merge['Date'] should already be datetime from steps above

    # Perform an outer merge on Account_Number and Date
    # Using suffixes to distinguish columns from merged_initial and bank_clean_merge
    combined = pd.merge(merged_initial, bank_clean_merge, on=['Account_Number', 'Date', 'Source_Type'], how='outer', suffixes=('_other', '_bank'))

    # Consolidate columns, prioritizing bank data where available
    # Handle cases where suffixes might not exist if one side of merge had no data for a given Account_Number/Date/Source_Type combination
    combined['Description'] = combined['Description_bank'].fillna(combined['Description_other'])
    combined['Amount'] = combined['Amount_bank'].fillna(combined['Amount_other'])
    combined['Balance'] = combined['Balance_bank'].fillna(combined['Balance_other']) # Note: Balance from bank is actual balance, from mpesa is running balance, bills is None
    # Source_Type is merged 'on' so it should be fine

    # Drop the original suffixed columns
    combined.drop(columns=['Description_other', 'Description_bank', 'Amount_other', 'Amount_bank', 'Balance_other', 'Balance_bank'], errors='ignore', inplace=True)

else:
    # If bank_clean_merge was empty, combined is just merged_initial
    combined = merged_initial.copy()
    print("Warning: Bank data is empty. Combined dataframe contains only MPESA and Utility Bill data.")


# ✅ Step 10: Save Final Dataset
# Sort by date if a consolidated date column exists or is created
if 'Date' in combined.columns:
    combined = combined.sort_values('Date').reset_index(drop=True)

# For now, let's save the combined dataframe as is, assuming the merge ran.
if not combined.empty:
    combined.to_csv('/content/combined_financial_data.csv', index=False)
    print("✅ Combined data saved to '/content/combined_financial_data.csv'")
    print("\nCombined DataFrame head:")
    display(combined.head())
    print("\nCombined DataFrame columns:")
    print(combined.columns.tolist())
else:
     print("Error: Combined DataFrame is empty after processing.")

✅ Combined data saved to '/content/combined_financial_data.csv'

Combined DataFrame head:


/tmp/ipython-input-4170840862.py:218: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_initial = pd.concat([mpesa_clean, bills_clean], ignore_index=True)


,Account_Number,Date,Description,Amount,Balance,Source_Type
0,USER_1,2024-08-01,nairobi water,3298.96,NaN,UTILITY_BILL
1,USER_8,2024-08-01,nairobi water,2271.97,NaN,UTILITY_BILL
2,USER_8,2024-08-01,kplc,9119.22,NaN,UTILITY_BILL
3,USER_5,2024-08-01,kplc,9548.40,NaN,UTILITY_BILL
4,USER_5,2024-08-01,nairobi water,1708.92,NaN,UTILITY_BILL



Combined DataFrame columns:
['Account_Number', 'Date', 'Description', 'Amount', 'Balance', 'Source_Type']


In [10]:
import fitz # PyMuPDF
import os

# Correcting the extraction path to match where the files were unzipped
extract_path = "/content/unzipped_pdfs"

# Find the first PDF file in the extracted directory
pdf_files = [f for f in os.listdir(extract_path) if f.endswith(".pdf")]

if pdf_files:
    first_pdf_path = os.path.join(extract_path, pdf_files[0])
    doc = fitz.open(first_pdf_path)
    text = doc[0].get_text()
    print("--- Extracted Text from First PDF ---")
    print(text)
    print("-------------------------------------")
else:
    print(f"No PDF files found in {extract_path}")

--- Extracted Text from First PDF ---
Bank Statement
2024
Bank Statement
Account Statement for Esther Wambui
Account Number: 7890-1234-5678-9012
Period: 01/01/2024 to 31/12/2024
Branch: Eldoret
Currency: KES
Date
Description
Details
Debit
(KES)
Credit
(KES)
Balance
(KES)
01/01/2024
Opening Balance
7,000.00
05/01/2024
Salary Credit
Hospital Salary
60,000.00
67,000.00
10/01/2024
M-Pesa Paybill
Rent
18,000.00
49,000.00
15/01/2024
Loan Repayment
Mobile Loan
4,000.00
45,000.00
20/01/2024
Savings Transfer
Savings A/C
5,000.00
40,000.00
10/02/2024
Salary Credit
Delayed Salary
60,000.00
100,000.00
05/05/2024
M-Pesa Paybill
Medical Training
15,000.00
85,000.00
10/05/2024
M-Pesa Transfer
Side Hustle
10,000.00
95,000.00
15/05/2024
M-Pesa Transfer
Unexplained Payment
20,000.00
75,000.00
20/05/2024
Savings Reversal
Cash Need
5,000.00
80,000.00
31/12/2024
Closing Balance
80,000.00
1 of 1

-------------------------------------


In [11]:
import pandas as pd
import zipfile
import os
from pathlib import Path

# File paths
zip_path = 'fwdmockbankstatements.zip'
csv1_path = 'synthetic_mpesa_transactions.csv'
csv2_path = 'utility_bills_with_mpesa.csv'
extract_dir = 'unzipped_statements'

# 1. Extract ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# 2. Load CSVs
mpesa_df = pd.read_csv(csv1_path)
bills_df = pd.read_csv(csv2_path)

# 3. Clean CSVs
mpesa_df.drop_duplicates(inplace=True)
bills_df.dropna(subset=['Final_Amount_KSh'], inplace=True)

# 4. Merge
common_cols = list(set(mpesa_df.columns) & set(bills_df.columns))
if common_cols:
    merged_df = pd.merge(mpesa_df, bills_df, on=common_cols[0], how='outer')
else:
    merged_df = pd.concat([mpesa_df, bills_df], axis=0)

# 5. Show PDF file names
pdf_files = list(Path(extract_dir).glob('*.pdf'))
print("PDFs ready for parsing:")
for f in pdf_files:
    print(" -", f.name)

# 6. Save merged result
merged_df.to_csv("merged_data_output.csv", index=False)
print("\nMerged data saved as merged_data_output.csv")


PDFs ready for parsing:
 - Statements (7).pdf
 - Statements (9).pdf
 - Statements (8).pdf
 - Statements (10).pdf
 - Statements (3).pdf
 - Statements (5).pdf
 - Statements (6).pdf
 - Statements (1).pdf
 - Statements (4).pdf
 - Statements (2).pdf

Merged data saved as merged_data_output.csv


In [12]:
# 🛠️ Step 1: Install Required Libraries
!pip install pandas PyMuPDF

# 📦 Step 2: Import Libraries
import pandas as pd
import zipfile
import os
from pathlib import Path
import re
import fitz  # PyMuPDF for PDF parsing

# 📁 Step 3: Mount Google Drive (if your files are there)
# from google.colab import drive
# drive.mount('/content/drive')

# 📂 Step 4: Set Paths to Your Files (update if needed)
mpesa_csv = '/content/synthetic_mpesa_transactions.csv'
bills_csv = '/content/utility_bills_with_mpesa.csv'
bank_zip = '/content/fwdmockbankstatements.zip'
extract_dir = '/content/unzipped_pdfs'

# 📤 Step 5: Extract Bank Statements from ZIP
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(bank_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# ✅ Step 6: Clean MPESA Transactions
mpesa = pd.read_csv(mpesa_csv)
mpesa.drop_duplicates(inplace=True)
# Removed string operations from numeric columns
mpesa['Transaction_Amount'] = mpesa['Transaction_Amount'].replace('[^\d.]', '', regex=True).astype(float)
mpesa['Balance'] = mpesa['Balance'].replace('[^\d.]', '', regex=True).astype(float)
# Removed string operation from Wallet_Impact
# mpesa['Wallet_Impact'] = mpesa['Wallet_Impact'].str.lower().str.strip() # Wallet_Impact is numeric
mpesa['Date'] = pd.to_datetime(mpesa['Date'], errors='coerce')
mpesa['Source_Type'] = 'MPESA'
# Rename 'Account' to 'Account_Number' for consistency before merging
if 'Account' in mpesa.columns:
    mpesa.rename(columns={"Account": "Account_Number"}, inplace=True)


# ✅ Step 7: Clean Utility Bills
bills = pd.read_csv(bills_csv)
bills.dropna(subset=['Final_Amount_KSh'], inplace=True)
bills['Final_Amount_KSh'] = bills['Final_Amount_KSh'].replace('[^\d.]', '', regex=True).astype(float)
# Assuming 'Provider' column is intended as the description and is string type
if 'Provider' in bills.columns:
    # bills['Bill_Type'] = bills['Bill_Type'].str.lower().str.strip() # Renamed to Description below
    bills.rename(columns={'Provider': 'Description'}, inplace=True) # Renaming Provider to Description
    if pd.api.types.is_object_dtype(bills['Description']) or pd.api.types.is_string_dtype(bills['Description']):
         bills['Description'] = bills['Description'].str.lower().str.strip()
    else:
         print("Warning: 'Description' column in utility bills is not a string type. Skipping string cleaning.")

bills['Date'] = pd.to_datetime(bills['Billing_Date'], errors='coerce') # Using Billing_Date as the Date for utility bills
# Assuming Balance_After is not present or needs to be calculated later
# bills.rename(columns={'Final_Amount_KSh': 'Amount', 'Balance_After': 'Balance', 'Bill_Type': 'Description'}, inplace=True)
bills.rename(columns={'Final_Amount_KSh': 'Amount'}, inplace=True)
bills['Source_Type'] = 'UTILITY_BILL'
# Rename 'User_ID' to 'Account_Number' for consistency before merging
if 'User_ID' in bills.columns:
    bills.rename(columns={"User_ID": "Account_Number"}, inplace=True)


# ✅ Step 8: Parse Bank PDFs (Using the corrected parsing logic from BWA3Kxh_ZOl0)
# Note: The parse_pdf_bank_statements function needs to be defined or copied here
# to be used within this cell if you want to run this cell independently after fixing parsing.
# For now, we'll assume the improved_parse_bank_statement_text from BWA3Kxh_ZOl0 is available and working.

# Replicating the parsing loop from BWA3Kxh_ZOl0 here for a self-contained cell
# This assumes improved_parse_bank_statement_text is defined above this point or in an earlier cell
all_data = []
if os.path.exists(extract_dir):
    for fname in os.listdir(extract_dir):
        if fname.endswith(".pdf"):
            doc_path = os.path.join(extract_dir, fname)
            try:
                doc = fitz.open(doc_path)
                text = ""
                for page in doc:
                    text += page.get_text()
                # Assuming improved_parse_bank_statement_text takes text as input and returns a DataFrame
                # Make sure this function is defined and correctly extracts Account_Number
                df = improved_parse_bank_statement_text(text) # Need to pass account number if extracted separately
                if not df.empty:
                    all_data.append(df)
            except Exception as e:
                print(f"Error processing {fname}: {e}")


# If parsing logic needs account number from header separately and not returned in df:
# You might need to adjust improved_parse_bank_statement_text to return account_number as well
# Or re-extract account number here and add to df

# Re-parsing approach to ensure account number is added if not done by the function
bank_transactions_list = []
if os.path.exists(extract_dir):
     for fname in os.listdir(extract_dir):
         if fname.endswith(".pdf"):
             doc_path = os.path.join(extract_dir, fname)
             try:
                 doc = fitz.open(doc_path)
                 text = ""
                 for page in doc:
                     text += page.get_text()

                 # Extract Account Number from header first
                 acct_match = re.search(r'Account Number:\s*([\d\-]+)', text)
                 account_number = acct_match.group(1).strip() if acct_match else "Unknown"

                 # Parse transactions, the function should ideally return transaction details without account number
                 # Or you modify the function to include account number
                 # Let's assume improved_parse_bank_statement_text parses rows and you add account_number here
                 # This requires modifying improved_parse_bank_statement_text to return a list of transaction dicts or similar
                 # instead of a DataFrame if account number is added outside.
                 # A better approach is to modify the function itself to include account_number.

                 # Assuming improved_parse_bank_statement_text is modified to return a DataFrame with Account_Number
                 df = improved_parse_bank_statement_text(text)
                 if not df.empty:
                      bank_transactions_list.append(df)

             except Exception as e:
                 print(f"Error processing {fname}: {e}")


if bank_transactions_list:
    bank_df = pd.concat(bank_transactions_list, ignore_index=True)
    bank_df['Source_Type'] = 'BANK_STATEMENT'
    # Ensure 'Date' column in bank_df is datetime
    if 'Date' in bank_df.columns and not pd.api.types.is_datetime64_any_dtype(bank_df['Date']):
        bank_df['Date'] = pd.to_datetime(bank_df['Date'], errors='coerce').dt.date # Keep as date objects for consistency with utility bills initially

    # Rename 'Account_Number' if the parsing function created a different name
    # if 'Account_Number' not in bank_df.columns and 'Parsed_Account' in bank_df.columns:
    #      bank_df.rename(columns={'Parsed_Account': 'Account_Number'}, inplace=True)

else:
    bank_df = pd.DataFrame()
    print("Warning: bank_df is empty. PDF parsing may have failed.")


# ✅ Step 9: Combine All Sources

# Ensure common columns exist and have appropriate names before merging
common_cols_initial = ['Account_Number', 'Date', 'Description', 'Amount', 'Balance', 'Source_Type']

# Select and rename columns for merging
mpesa_clean = mpesa[mpesa.columns.intersection(['Account_Number', 'Date', 'Transaction_Type', 'Transaction_Amount', 'Balance', 'Source_Type'])].copy()
mpesa_clean.rename(columns={'Transaction_Type': 'Description', 'Transaction_Amount': 'Amount'}, inplace=True)
mpesa_clean = mpesa_clean[common_cols_initial].copy() # Reindex to ensure consistent column order and drop extras


bills_clean = bills[bills.columns.intersection(['Account_Number', 'Date', 'Description', 'Amount', 'Source_Type'])].copy()
# Bills data doesn't have a 'Balance' column directly equivalent to account balance
bills_clean['Balance'] = None # Add a placeholder Balance column
bills_clean = bills_clean[common_cols_initial].copy() # Reindex


# Ensure bank_df has the expected columns if populated
if not bank_df.empty:
    bank_clean = bank_df[bank_df.columns.intersection(['Account_Number', 'Date', 'Description', 'Debit', 'Credit', 'Balance', 'Source_Type'])].copy()
    # Decide how to represent Debit/Credit as a single 'Amount' for merging if needed, or keep separate
    # For initial merge, let's keep Debit/Credit separate and 'Amount' from others.
    # We'll need to align these later or use a different merge strategy.

    # Option 1: Create 'Amount' from Debit/Credit (simplistic, might lose info)
    # bank_clean['Amount'] = bank_clean['Credit'].fillna(0) - bank_clean['Debit'].fillna(0) # Or just use Credit as income, Debit as expense

    # Option 2: Keep Debit/Credit separate and handle during feature engineering
    # For merging, we might only need Account_Number and Date
    # Let's proceed with merging on Account_Number and Date where possible

    # Select columns for bank_clean for merging
    bank_clean_merge = bank_df[bank_df.columns.intersection(['Account_Number', 'Date', 'Source_Type', 'Debit', 'Credit', 'Balance', 'Description'])].copy()
    # Ensure Date is datetime for merging
    if 'Date' in bank_clean_merge.columns and not pd.api.types.is_datetime64_any_dtype(bank_clean_merge['Date']):
         bank_clean_merge['Date'] = pd.to_datetime(bank_clean_merge['Date'], errors='coerce')
    # Ensure Account_Number exists
    if 'Account_Number' not in bank_clean_merge.columns:
         print("Error: 'Account_Number' column is missing in bank_clean_merge. Cannot merge.")
         bank_clean_merge = pd.DataFrame() # Make it empty to prevent merge errors

else:
    bank_clean_merge = pd.DataFrame()
    print("Warning: bank_clean_merge is empty as bank_df was empty.")


# Perform Merges
# Start by merging mpesa and bills on Account_Number and Date
# Note: This merge strategy might not be ideal if timestamps don't match exactly.
# Consider merging on Account_Number and then handling time-based relationships during feature engineering.

# Let's try merging on Account_Number first, then handle time relationships later.
merged_initial = pd.concat([mpesa_clean, bills_clean], ignore_index=True)

# Now merge with bank data. This is tricky due to different structures (Debit/Credit vs single Amount)
# A full outer merge on Account_Number seems appropriate to keep all records.
# We'll use suffixes to distinguish columns.

if not bank_clean_merge.empty:
    combined = pd.merge(merged_initial, bank_clean_merge, on='Account_Number', how='outer', suffixes=('_other', '_bank'))
else:
    combined = merged_initial.copy() # If bank_df is empty, just use mpesa and bills data


# Handle Date column: Consolidate dates if possible, or keep separate and handle during analysis
# This part needs careful consideration based on how you want to use date information.
# For simplicity for now, let's assume we have Date_other and Date_bank and handle them later.

# ✅ Step 10: Save Final Dataset
# Fill NaN values created by outer merge (optional, decide based on analysis needs)
# combined.fillna(0, inplace=True) # Be cautious with filling dates or specific columns with 0

# Sort by date if a consolidated date column exists or is created
# If using Date_other and Date_bank, you might need a different sorting strategy or handle time in feature engineering.

# For now, let's save the combined dataframe as is, assuming the merge ran.
if not combined.empty:
    combined.to_csv('/content/combined_financial_data.csv', index=False)
    print("✅ Combined data saved to '/content/combined_financial_data.csv'")
    print("\nCombined DataFrame head:")
    display(combined.head())
    print("\nCombined DataFrame columns:")
    print(combined.columns.tolist())
else:
     print("Error: Combined DataFrame is empty. Merging failed.")

✅ Combined data saved to '/content/combined_financial_data.csv'

Combined DataFrame head:


/tmp/ipython-input-709660396.py:195: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_initial = pd.concat([mpesa_clean, bills_clean], ignore_index=True)


,Account_Number,Date,Description,Amount,Balance,Source_Type
0,USER_9,2025-07-01 00:00:00,Pay Bill,0.0,1000.0,MPESA
1,USER_10,2025-07-01 00:02:00,Withdraw,0.0,1000.0,MPESA
2,USER_10,2025-07-01 00:04:00,Withdraw,0.0,1000.0,MPESA
3,USER_7,2025-07-01 00:04:00,Send Money,0.0,1000.0,MPESA
4,USER_4,2025-07-01 00:06:00,Deposit,0.0,1000.0,MPESA



Combined DataFrame columns:
['Account_Number', 'Date', 'Description', 'Amount', 'Balance', 'Source_Type']


In [13]:
# 🛠️ Step 1: Install Required Libraries
!pip install pandas PyMuPDF

# 📦 Step 2: Import Libraries
import pandas as pd
import zipfile
import os
from pathlib import Path
import re
import fitz  # PyMuPDF for PDF parsing

# 📁 Step 3: Mount Google Drive (if your files are there)
# from google.colab import drive
# drive.mount('/content/drive')

# 📂 Step 4: Set Paths to Your Files (update if needed)
mpesa_csv = '/content/synthetic_mpesa_transactions.csv'
bills_csv = '/content/utility_bills_with_mpesa.csv'
bank_zip = '/content/fwdmockbankstatements.zip'
extract_dir = '/content/unzipped_pdfs'

# 📤 Step 5: Extract Bank Statements from ZIP
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(bank_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# ✅ Step 6: Clean MPESA Transactions
mpesa = pd.read_csv(mpesa_csv)
mpesa.drop_duplicates(inplace=True)
# Removed string operations from numeric columns
mpesa['Transaction_Amount'] = mpesa['Transaction_Amount'].replace('[^\d.]', '', regex=True).astype(float)
mpesa['Balance'] = mpesa['Balance'].replace('[^\d.]', '', regex=True).astype(float)
# Removed string operation from Wallet_Impact
# mpesa['Wallet_Impact'] = mpesa['Wallet_Impact'].str.lower().str.strip() # Wallet_Impact is numeric
mpesa['Date'] = pd.to_datetime(mpesa['Date'], errors='coerce')
mpesa['Source_Type'] = 'MPESA'
# Rename 'Account' to 'Account_Number' for consistency before merging
if 'Account' in mpesa.columns:
    mpesa.rename(columns={"Account": "Account_Number"}, inplace=True)


# ✅ Step 7: Clean Utility Bills
bills = pd.read_csv(bills_csv)
bills.dropna(subset=['Final_Amount_KSh'], inplace=True)
bills['Final_Amount_KSh'] = bills['Final_Amount_KSh'].replace('[^\d.]', '', regex=True).astype(float)
# Assuming 'Provider' column is intended as the description and is string type
if 'Provider' in bills.columns:
    # bills['Bill_Type'] = bills['Bill_Type'].str.lower().str.strip() # Renamed to Description below
    bills.rename(columns={'Provider': 'Description'}, inplace=True) # Renaming Provider to Description
    if pd.api.types.is_object_dtype(bills['Description']) or pd.api.types.is_string_dtype(bills['Description']):
         bills['Description'] = bills['Description'].str.lower().str.strip()
    else:
         print("Warning: 'Description' column in utility bills is not a string type. Skipping string cleaning.")

bills['Date'] = pd.to_datetime(bills['Billing_Date'], errors='coerce') # Using Billing_Date as the Date for utility bills
# Assuming Balance_After is not present or needs to be calculated later
# bills.rename(columns={'Final_Amount_KSh': 'Amount', 'Balance_After': 'Balance', 'Bill_Type': 'Description'}, inplace=True)
bills.rename(columns={'Final_Amount_KSh': 'Amount'}, inplace=True)
bills['Source_Type'] = 'UTILITY_BILL'
# Rename 'User_ID' to 'Account_Number' for consistency before merging
if 'User_ID' in bills.columns:
    bills.rename(columns={"User_ID": "Account_Number"}, inplace=True)


# ✅ Step 8: Parse Bank PDFs (Using the corrected parsing logic from BWA3Kxh_ZOl0)
# Note: The parse_pdf_bank_statements function needs to be defined or copied here
# to be used within this cell if you want to run this cell independently after fixing parsing.
# For now, we'll assume the improved_parse_bank_statement_text from BWA3Kxh_ZOl0 is available and working.

# Replicating the parsing loop from BWA3Kxh_ZOl0 here for a self-contained cell
# This assumes improved_parse_bank_statement_text is defined above this point or in an earlier cell
all_data = []
if os.path.exists(extract_dir):
    for fname in os.listdir(extract_dir):
        if fname.endswith(".pdf"):
            doc_path = os.path.join(extract_dir, fname)
            try:
                doc = fitz.open(doc_path)
                text = ""
                for page in doc:
                    text += page.get_text()
                # Assuming improved_parse_bank_statement_text takes text as input and returns a DataFrame
                # Make sure this function is defined and correctly extracts Account_Number
                df = improved_parse_bank_statement_text(text) # Need to pass account number if extracted separately
                if not df.empty:
                    all_data.append(df)
            except Exception as e:
                print(f"Error processing {fname}: {e}")


# If parsing logic needs account number from header separately and not returned in df:
# You might need to adjust improved_parse_bank_statement_text to return account_number as well
# Or re-extract account number here and add to df

# Re-parsing approach to ensure account number is added if not done by the function
bank_transactions_list = []
if os.path.exists(extract_dir):
     for fname in os.listdir(extract_dir):
         if fname.endswith(".pdf"):
             doc_path = os.path.join(extract_dir, fname)
             try:
                 doc = fitz.open(doc_path)
                 text = ""
                 for page in doc:
                     text += page.get_text()

                 # Extract Account Number from header first
                 acct_match = re.search(r'Account Number:\s*([\d\-]+)', text)
                 account_number = acct_match.group(1).strip() if acct_match else "Unknown"

                 # Parse transactions, the function should ideally return transaction details without account number
                 # Or you modify the function to include account number
                 # Let's assume improved_parse_bank_statement_text parses rows and you add account_number here
                 # This requires modifying improved_parse_bank_statement_text to return a list of transaction dicts or similar
                 # instead of a DataFrame if account number is added outside.
                 # A better approach is to modify the function itself to include account_number.

                 # Assuming improved_parse_bank_statement_text is modified to return a DataFrame with Account_Number
                 df = improved_parse_bank_statement_text(text)
                 if not df.empty:
                      bank_transactions_list.append(df)

             except Exception as e:
                 print(f"Error processing {fname}: {e}")


if bank_transactions_list:
    bank_df = pd.concat(bank_transactions_list, ignore_index=True)
    bank_df['Source_Type'] = 'BANK_STATEMENT'
    # Ensure 'Date' column in bank_df is datetime
    if 'Date' in bank_df.columns and not pd.api.types.is_datetime64_any_dtype(bank_df['Date']):
        bank_df['Date'] = pd.to_datetime(bank_df['Date'], errors='coerce').dt.date # Keep as date objects for consistency with utility bills initially

    # Rename 'Account_Number' if the parsing function created a different name
    # if 'Account_Number' not in bank_df.columns and 'Parsed_Account' in bank_df.columns:
    #      bank_df.rename(columns={'Parsed_Account': 'Account_Number'}, inplace=True)

else:
    bank_df = pd.DataFrame()
    print("Warning: bank_df is empty. PDF parsing may have failed.")


# ✅ Step 9: Combine All Sources

# Ensure common columns exist and have appropriate names before merging
common_cols_initial = ['Account_Number', 'Date', 'Description', 'Amount', 'Balance', 'Source_Type']

# Select and rename columns for merging
mpesa_clean = mpesa[mpesa.columns.intersection(['Account_Number', 'Date', 'Transaction_Type', 'Transaction_Amount', 'Balance', 'Source_Type'])].copy()
mpesa_clean.rename(columns={'Transaction_Type': 'Description', 'Transaction_Amount': 'Amount'}, inplace=True)
mpesa_clean = mpesa_clean[common_cols_initial].copy() # Reindex to ensure consistent column order and drop extras


bills_clean = bills[bills.columns.intersection(['Account_Number', 'Date', 'Description', 'Amount', 'Source_Type'])].copy()
# Bills data doesn't have a 'Balance' column directly equivalent to account balance
bills_clean['Balance'] = None # Add a placeholder Balance column
bills_clean = bills_clean[common_cols_initial].copy() # Reindex


# Ensure bank_df has the expected columns if populated
if not bank_df.empty:
    bank_clean = bank_df[bank_df.columns.intersection(['Account_Number', 'Date', 'Description', 'Debit', 'Credit', 'Balance', 'Source_Type'])].copy()
    # Decide how to represent Debit/Credit as a single 'Amount' for merging if needed, or keep separate
    # For initial merge, let's keep Debit/Credit separate and 'Amount' from others.
    # We'll need to align these later or use a different merge strategy.

    # Option 1: Create 'Amount' from Debit/Credit (simplistic, might lose info)
    # bank_clean['Amount'] = bank_clean['Credit'].fillna(0) - bank_clean['Debit'].fillna(0) # Or just use Credit as income, Debit as expense

    # Option 2: Keep Debit/Credit separate and handle during feature engineering
    # For merging, we might only need Account_Number and Date
    # Let's proceed with merging on Account_Number and Date where possible

    # Select columns for bank_clean for merging
    bank_clean_merge = bank_df[bank_df.columns.intersection(['Account_Number', 'Date', 'Source_Type', 'Debit', 'Credit', 'Balance', 'Description'])].copy()
    # Ensure Date is datetime for merging
    if 'Date' in bank_clean_merge.columns and not pd.api.types.is_datetime64_any_dtype(bank_clean_merge['Date']):
         bank_clean_merge['Date'] = pd.to_datetime(bank_clean_merge['Date'], errors='coerce')
    # Ensure Account_Number exists
    if 'Account_Number' not in bank_clean_merge.columns:
         print("Error: 'Account_Number' column is missing in bank_clean_merge. Cannot merge.")
         bank_clean_merge = pd.DataFrame() # Make it empty to prevent merge errors

else:
    bank_clean_merge = pd.DataFrame()
    print("Warning: bank_clean_merge is empty as bank_df was empty.")


# Perform Merges
# Start by merging mpesa and bills on Account_Number and Date
# Note: This merge strategy might not be ideal if timestamps don't match exactly.
# Consider merging on Account_Number and then handling time-based relationships during feature engineering.

# Let's try merging on Account_Number first, then handle time relationships later.
merged_initial = pd.concat([mpesa_clean, bills_clean], ignore_index=True)

# Now merge with bank data. This is tricky due to different structures (Debit/Credit vs single Amount)
# A full outer merge on Account_Number seems appropriate to keep all records.
# We'll use suffixes to distinguish columns.

if not bank_clean_merge.empty:
    combined = pd.merge(merged_initial, bank_clean_merge, on='Account_Number', how='outer', suffixes=('_other', '_bank'))
else:
    combined = merged_initial.copy() # If bank_df is empty, just use mpesa and bills data


# Handle Date column: Consolidate dates if possible, or keep separate and handle during analysis
# This part needs careful consideration based on how you want to use date information.
# For simplicity for now, let's assume we have Date_other and Date_bank and handle them later.

# ✅ Step 10: Save Final Dataset
# Fill NaN values created by outer merge (optional, decide based on analysis needs)
# combined.fillna(0, inplace=True) # Be cautious with filling dates or specific columns with 0

# Sort by date if a consolidated date column exists or is created
# If using Date_other and Date_bank, you might need a different sorting strategy or handle time in feature engineering.

# For now, let's save the combined dataframe as is, assuming the merge ran.
if not combined.empty:
    combined.to_csv('/content/combined_financial_data.csv', index=False)
    print("✅ Combined data saved to '/content/combined_financial_data.csv'")
    print("\nCombined DataFrame head:")
    display(combined.head())
    print("\nCombined DataFrame columns:")
    print(combined.columns.tolist())
else:
     print("Error: Combined DataFrame is empty. Merging failed.")

✅ Combined data saved to '/content/combined_financial_data.csv'

Combined DataFrame head:


/tmp/ipython-input-709660396.py:195: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_initial = pd.concat([mpesa_clean, bills_clean], ignore_index=True)


,Account_Number,Date,Description,Amount,Balance,Source_Type
0,USER_9,2025-07-01 00:00:00,Pay Bill,0.0,1000.0,MPESA
1,USER_10,2025-07-01 00:02:00,Withdraw,0.0,1000.0,MPESA
2,USER_10,2025-07-01 00:04:00,Withdraw,0.0,1000.0,MPESA
3,USER_7,2025-07-01 00:04:00,Send Money,0.0,1000.0,MPESA
4,USER_4,2025-07-01 00:06:00,Deposit,0.0,1000.0,MPESA



Combined DataFrame columns:
['Account_Number', 'Date', 'Description', 'Amount', 'Balance', 'Source_Type']


In [14]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Load the cleaned data
@st.cache_data
def load_data():
    df = pd.read_csv("cleaned_credit_data.csv")
    df['Date'] = pd.to_datetime(df['Date'])
    df['Source_Type'] = LabelEncoder().fit_transform(df['Source_Type'])
    df['Description'] = df['Description'].astype(str).str.lower()
    return df

df = load_data()

# Add a target column for demonstration
if 'Risk_Label' not in df.columns:
    df['Risk_Label'] = df['Balance'].apply(lambda x: 0 if x < 500 else 1)

# Title
st.title("HEVA Credit Intelligence Dashboard")
st.markdown("Analyze MPESA, Utility, and Bank data for credit scoring insights.")

# Sidebar filters
st.sidebar.header("Filter Options")
source_filter = st.sidebar.multiselect("Select Source Type", options=df['Source_Type'].unique(), default=df['Source_Type'].unique())
amount_range = st.sidebar.slider("Transaction Amount Range", float(df['Amount'].min()), float(df['Amount'].max()), (float(df['Amount'].min()), float(df['Amount'].max())))

# Apply filters
filtered_df = df[(df['Source_Type'].isin(source_filter)) &
                 (df['Amount'] >= amount_range[0]) & (df['Amount'] <= amount_range[1])]

# Show data sample
st.subheader("Filtered Transactions")
st.dataframe(filtered_df.head(20))

# Plot 1: Amount over time
st.subheader("Transaction Amounts Over Time")
fig1, ax1 = plt.subplots()
filtered_df.groupby('Date')['Amount'].sum().plot(ax=ax1)
ax1.set_ylabel("Total Amount")
ax1.set_title("Daily Transaction Amounts")
st.pyplot(fig1)

# Plot 2: Source Type Breakdown
st.subheader("Transaction Source Breakdown")
fig2, ax2 = plt.subplots()
sns.countplot(data=filtered_df, x='Source_Type', ax=ax2)
ax2.set_title("Transactions by Source")
st.pyplot(fig2)

# Summary stats
st.subheader("Summary Statistics")
st.write(filtered_df.describe())

# Credit Risk Predictor
st.subheader("📊 Predict Credit Risk")

input_amount = st.number_input("Transaction Amount", value=1000.0)
input_balance = st.number_input("Account Balance", value=500.0)
input_source = st.selectbox("Source Type", options=sorted(df['Source_Type'].unique()))

if st.button("Predict Risk"):
    # Prepare model training
    features = ['Amount', 'Balance', 'Source_Type']
    X = df[features]
    y = df['Risk_Label']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_scaled, y)

    # Predict
    user_input = pd.DataFrame([[input_amount, input_balance, input_source]], columns=features)
    user_scaled = scaler.transform(user_input)
    prediction = model.predict(user_scaled)[0]
    prediction_proba = model.predict_proba(user_scaled)[0]

    # Output result
    st.markdown(f"### Risk Prediction: {'🟢 Low Risk' if prediction == 1 else '🔴 High Risk'}")
    st.write(f"Confidence: {round(np.max(prediction_proba)*100, 2)}%")

# Footer
st.markdown("---")
st.caption("Built with ❤️ by HEVA Credit Analytics")


ModuleNotFoundError: No module named 'streamlit'